# **INTRODUCTIONS**

---

This flow is added because of a depencency to ECDC_VARIANTS



1. **[Tables](#tables)**
1. **[Datatino configurations](#DATATINO-CONFIGURATIONS)**


# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb"
    ]
}
```

# **TABLES**
---

### STAGING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSSTAGE_MASTERDATA_VARIANTS]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSSTAGE_MASTERDATA_VARIANTS
  START WITH 1
  INCREMENT BY 1;
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[MASTERDATA_VARIANTS]'))
BEGIN
  CREATE TABLE [VWSSTAGE].[MASTERDATA_VARIANTS] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [VARIANT_CODE] VARCHAR (100) NULL,
    [SHOWS_ON_VWS_DASHBOARD] VARCHAR (100) NULL,
    [LABEL_EN] VARCHAR (100) NULL,
    [LABEL_NL] VARCHAR (100) NULL,
    [SORT_ORDER] VARCHAR (100) NULL,
    [IS_SUBVARIANT_OF] VARCHAR (100) NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (GETDATE()) NULL
  );
END
ELSE
BEGIN
	IF NOT EXISTS(SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (
            N'LABEL_EN', 
            N'LABEL_NL', 
            N'SORT_ORDER'
            )
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[MASTERDATA_VARIANTS]'))
	BEGIN
		ALTER TABLE [VWSSTAGE].[MASTERDATA_VARIANTS] 
		ADD 
            [LABEL_EN] VARCHAR (100) NULL,
            [LABEL_NL] VARCHAR (100) NULL,
            [SORT_ORDER] VARCHAR (100) NULL;
	END

  IF EXISTS(SELECT * FROM SYS.COLUMNS 
    WHERE [NAME] IN (
        N'ECDC_CATEGORY', 
        N'WHO_CATEGORY', 
        N'VARIANT_NAME', 
        N'LABEL'
        )
    AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[MASTERDATA_VARIANTS]'))
    BEGIN
    ALTER TABLE [VWSSTAGE].[MASTERDATA_VARIANTS] 
		        DROP COLUMN ECDC_CATEGORY,
            WHO_CATEGORY,
            VARIANT_NAME,
            LABEL;
	  END

END
GO

### STATICS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

-- 1) CREATE TABLE(S).....

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[MASTERDATA_VARIANTS]'))
BEGIN
  CREATE TABLE [VWSSTATIC].[MASTERDATA_VARIANTS] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [VARIANT_CODE] VARCHAR (100) NULL,
    [SHOWS_ON_VWS_DASHBOARD]  BIT DEFAULT(0) NULL,
    [LABEL_EN] VARCHAR (100) NULL,
    [LABEL_NL] VARCHAR (100) NULL,
    [SORT_ORDER] INT NULL,
    [IS_SUBVARIANT_OF] VARCHAR (100) NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (GETDATE()) NULL
  );
END
ELSE
BEGIN
	IF NOT EXISTS(SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (
            N'LABEL_EN', 
            N'LABEL_NL', 
            N'SORT_ORDER'
            )
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[MASTERDATA_VARIANTS]'))
	BEGIN
		ALTER TABLE [VWSSTATIC].[MASTERDATA_VARIANTS] 
		ADD 
            [LABEL_EN] VARCHAR (100) NULL,
            [LABEL_NL] VARCHAR (100) NULL,
            [SORT_ORDER] INT NULL;
	END

  IF EXISTS(SELECT * FROM SYS.COLUMNS 
    WHERE [NAME] IN (
        N'ECDC_CATEGORY', 
        N'WHO_CATEGORY', 
        N'VARIANT_NAME', 
        N'LABEL'
        )
    AND [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[MASTERDATA_VARIANTS]'))
    BEGIN
    ALTER TABLE [VWSSTATIC].[MASTERDATA_VARIANTS] 
              DROP COLUMN ECDC_CATEGORY,
              WHO_CATEGORY,
              VARIANT_NAME,
              LABEL;
	  END

END

GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_MASTERDATA_VARIANTS]
  ON [VWSSTATIC].[MASTERDATA_VARIANTS]
GO

CREATE INDEX [NCIX_DLI_MASTERDATA_VARIANTS]
  ON [VWSSTATIC].[MASTERDATA_VARIANTS] (
    [DATE_LAST_INSERTED]
  )
GO

# **STORED PROCEDURES**
---

### STAGE &rarr; STATIC MAPPING

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [DBO].[SP_MASTERDATA_VARIANTS_STAGING_TO_STATIC]
AS
BEGIN
        INSERT INTO [VWSSTATIC].[MASTERDATA_VARIANTS] (
                [VARIANT_CODE],
                [SHOWS_ON_VWS_DASHBOARD],
                [LABEL_EN],
                [LABEL_NL],
                [SORT_ORDER],
                [IS_SUBVARIANT_OF]
        )
        SELECT
                [VARIANT_CODE],
                [SHOWS_ON_VWS_DASHBOARD],
                [LABEL_EN],
                [LABEL_NL],
                [SORT_ORDER],
                [IS_SUBVARIANT_OF]
        FROM [VWSSTAGE].[MASTERDATA_VARIANTS]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MASTERDATA_VARIANTS])
END;
GO

# **DATATINO CONFIGURATIONS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'MASTERDATA_VARIANTS',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;


    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 1
        WHEN 'acceptance' THEN 1
        ELSE 1
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @schedule = '0 12 * * FRI', -- AT 14:00 PM (DUTCH SUMMER TIME), ONLY ON FRIDAYS  
        @active = @is_active;

    -- 2) UPSERT SOURCE(S).....
    DECLARE @source NVARCHAR(256),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    -- 2.1) SET ENVIRONMENTAL VARIABLES.....
    SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/masterdata_vws_varianten.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/masterdata_vws_varianten.csv'
        ELSE 'datafiles/STATIC/masterdata_vws_varianten.csv'
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'AzureBlob'
        WHEN 'acceptance' THEN 'AzureBlob'
        ELSE 'AzureBlob'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'N/A'
        WHEN 'acceptance' THEN 'N/A'
        ELSE 'N/A'
    END;

    -- 2.2) UPSERT TABLE SOURCE(S): STAGING......
    SET @source_name = CONCAT('SOURCE_', @workflow_name);
    SET @source_description = CONCAT('LOAD: ', @workflow_name);
    SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = 'Variant_code|Shows_on_dashboard|Label_EN|Label_NL|Sort_order|Is_subvariant_of',
        @target_columns = 'VARIANT_CODE|SHOWS_ON_VWS_DASHBOARD|LABEL_EN|LABEL_NL|SORT_ORDER|IS_SUBVARIANT_OF',
        @target_name = @target_name,
        @source_type = 'CsvFile',
        @location_type = @location_type,
        @delimiter_type = 'SemiColon',
        @security_profile= @security_profile;

    -- 2) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 2.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = CONCAT('PROCESS_', @workflow_name);
    SET @process_description = CONCAT('LOAD: ', @workflow_name);
    SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

### UPSERT STORED PROCEDURE STAGING TO STATIC

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- 1 UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @workflow_name NVARCHAR(50) = 'MASTERDATA_VARIANTS';
        

SET @source = 'DBO.SP_MASTERDATA_VARIANTS_STAGING_TO_STATIC';
SET @source_name = 'SOURCE_Staging_Masterdata_variants';
SET @source_description = 'Move masterdata variants data from stage table to static table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

-- 2) UPSERT STORED PROCEDURE PROCESS(S): STAGING TO STATIC......
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);


SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGING_TO_STATIC');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGING TO STATIC');
SET @process_source_name =  @source_name --CONCAT('SOURCE_SP_', @workflow_name, '_STAGING_TO_STATIC');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES)....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGING_TO_STATIC');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGING_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;